In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import geojson as gj

In [2]:
# Create DF for LAPD_Divisions_geojson
lapd_file = "../static/data/LAPD_Divisions.geojson"
lapd_df = gpd.read_file(lapd_file)

In [3]:
# lapd_df.head()

In [4]:
# Load json to dataframe
year = [2013,2014,2015,2016,2017,2018,2019]

yearArea_df = pd.DataFrame(columns = ['Year', 'LAPD_Area', 'Crime_Counts'])

# Loop through each year and append to a single dataframe

for y in year:
    crime_year = f"crime_{y}"
    file_to_load = f'../static/data/source/{crime_year}.json'
    
    crime_df = pd.read_json(file_to_load).T

    # Aggregate by Year and Area
    grouped_df = crime_df.groupby(['Year','LAPD_Area']).agg({'Crime_Type':['count']}) 
    grouped_df.columns = ['Crime_Counts'] 
    grouped_df.reset_index(level=['Year','LAPD_Area'],inplace=True)
    yearArea_df = yearArea_df.append(grouped_df, ignore_index = True)
    

In [5]:
# yearArea_df.head()

In [6]:
# Join grouped_df (crime) with LAPD_divsions - join by area
final_df = yearArea_df.merge(lapd_df, left_on='LAPD_Area', right_on='PREC')
# final_df.head(100)

In [7]:
# create function to output a geojson format
def data2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            gj.Feature(geometry=X["geometry"],
                       #gj.Point((X["LON"],X["LAT"])),  ## old code and no longer needed
                            properties=dict(crime_counts=X["Crime_Counts"]
                                           ,year=X["Year"])))
    df.apply(insert_features, axis=1)
    with open(filename, 'w', encoding='utf8') as fp:
        gj.dump(gj.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

In [8]:
# Create final crime GeoJson
outputFilePath = '../static/data/'
outputFileName = 'crime'
dispFileName = f'{outputFilePath}{outputFileName}.geojson'

data2geojson(final_df, dispFileName)